In [142]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

# 1. Load data

In [83]:
mnist = input_data.read_data_sets("./datasets/mnist_data/", one_hot=True)

Extracting ./datasets/mnist_data/train-images-idx3-ubyte.gz
Extracting ./datasets/mnist_data/train-labels-idx1-ubyte.gz
Extracting ./datasets/mnist_data/t10k-images-idx3-ubyte.gz
Extracting ./datasets/mnist_data/t10k-labels-idx1-ubyte.gz


# 2. Start Session

In [84]:
sess = tf.InteractiveSession()

# 3. Build Softmax regression

In [85]:
x = tf.placeholder(tf.float32, [None, 784])

In [86]:
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

In [87]:
y = tf.nn.softmax(tf.matmul(x, W) + b)
y.shape

TensorShape([Dimension(None), Dimension(10)])

In [88]:
y_ = tf.placeholder(tf.float32, [None, 10])
y_.get_shape()

TensorShape([Dimension(None), Dimension(10)])

In [89]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

In [90]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [91]:
sess.run(tf.global_variables_initializer())

In [92]:
for _ in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

In [93]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

In [94]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
accuracy.shape

TensorShape([])

In [95]:
sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels})

0.90740001

# 4. Build a multilayer Convolutional Network

In [118]:
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)
        
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [98]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

## First convolutional layer

In [119]:
with tf.name_scope('layer1'):
    W_conv1 = weight_variable([5, 5, 1, 32])
    b_conv1 = bias_variable([32])
    x_image = tf.reshape(x, [-1, 28, 28, 1])
    # We then convolve x_image with the weight tensor, add the bias, apply the ReLU function, and finally max pool
    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
    h_pool1 = max_pool_2x2(h_conv1)
    print("h_pool1.shape", h_pool1.shape)
    print("h_conv1.shape", h_conv1.shape)

('h_pool1.shape', TensorShape([Dimension(None), Dimension(14), Dimension(14), Dimension(32)]))
('h_conv1.shape', TensorShape([Dimension(None), Dimension(28), Dimension(28), Dimension(32)]))


## Second Convolutional Layer

In [120]:
with tf.name_scope('layer2'):
    W_conv2 = weight_variable([5, 5, 32, 64])
    b_conv2 = bias_variable([64])
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
    h_pool2 = max_pool_2x2(h_conv2)
    print(h_pool2.shape)

(?, 7, 7, 64)


# Densely Connected Layer

In [121]:
with tf.name_scope('layer3'):
    W_fc1 = weight_variable([7 * 7 * 64, 1024])
    b_fc1 = bias_variable([1024])
    h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
    print(h_fc1.shape)

(?, 1024)


# Dropout

In [160]:
with tf.name_scope("layer4_dropout"):
    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
    tf.summary.scalar('dropout_keep_probability', keep_prob)
    print(h_fc1_drop.shape)

(?, 1024)


# Readout Layer

In [161]:
with tf.name_scope("layer5_readout_layer"):
    W_fc2 = weight_variable([1024, 10])
    b_fc2 = bias_variable([10])
    y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2
    print(y_conv.shape)

(?, 10)


# Describe Loss Function

In [162]:
with tf.name_scope("cross_entropy"):
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
tf.summary.scalar('cross_entropy', cross_entropy)

<tf.Tensor 'cross_entropy_10:0' shape=() dtype=string>

# Train and evaluate

In [163]:
with tf.name_scope("train"):
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
with tf.name_scope("accuracy"):    
    correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
tf.summary.scalar("accuracy", accuracy)

<tf.Tensor 'accuracy_10:0' shape=() dtype=string>

In [164]:
# Merge all the summaries and write them out to
# /tmp/tensorflow/mnist/logs/mnist_with_summaries (by default)
merged = tf.summary.merge_all()
train_writer = tf.summary.FileWriter("./logs/mnist" + '/train', sess.graph)
test_writer = tf.summary.FileWriter("./logs/mnist" + '/test')

In [165]:
def feed_dict(train):
    """Make a TensorFlow feed_dict: maps data onto Tensor placeholders."""
    if train:
      xs, ys = mnist.train.next_batch(100, False)
      k = np.float32(0.5)
    else:
      xs, ys = mnist.test.images, mnist.test.labels
      k = np.float32(1.0)
    return {x: xs, y_: ys, keep_prob: k}

In [ ]:
# train the model and collect test summaries every 10 steps
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(1000):
        train_feed_dict = feed_dict(True)
        test_feed_dict = feed_dict(False)
        if i % 100 == 0:
#             summary, train_accuracy = sess.run([merged, accuracy], feed_dict=feed_dict(False))
#             test_writer.add_summary(summary, i)
            train_accuracy = accuracy.eval(feed_dict=train_feed_dict)
            test_accuracy = accuracy.eval(feed_dict=test_feed_dict)
            print("step {0}, test accuracy {1}, test_accuracy: {2}".format(i, train_accuracy, test_accuracy))
        sess.run(train_step, feed_dict=train_feed_dict)
               
    test_writer.close()
    train_writer.close()
    
    # Evaluate the accuracy
    test_accuracy = accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob:1.0})
    print('test accuracy: {0}'.format(test_accuracy))